In [25]:
using JuMP, Ipopt, FurutaPendulums

furuta = SimulatedFurutaPendulum()

# Define the objective function
function objective(ϕ, ϕdot, θ, θdot, u)
    # Define the objective function to be minimized
    # ϕ, ϕdot, θ, θdot = x
    # cost = abs(0.0018522 - 0.0018522 * cos(θ) - 0.00000387 * θdot^2)/0.0018522  # Define the cost function
    cost = 1-cos(θ)
    return cost
end

# Define the NMPC controller
function nmpc_controller(x0, N, dt)
    # dynamic parameters
    J = 1.54e-4
    M = 0
    ma = 0
    mp = 5.44e-3
    la = 4.3e-2
    lp = 6.46e-2
    fϕ = 0.0238
    fθ = 0.000008
    R = 0.13
    Ka = 0.03
    h = 0.01
    g = 9.81
    α = J+(M+ma/3+mp)*la^2
    β = (M+mp/3)*lp^2
    γ = (M+mp/2)*la*lp
    δ = (M+mp/2)*g*lp
    Ku = Ka / R

    # τϕm = Ku * u # Torque from motor (approximating that transients between voltage and current are fast)
    # τϕf = -fϕ * ϕdot
    # τϕ = τϕm + τϕf
    # # theta joint
    # τθ = -fθ * θdot

    #MPC model parameters
    n_states = 4
    n_inputs = 1

    # Create a JuMP optimization model
    model = Model(Ipopt.Optimizer)
    # set_attribute(model, "max_cpu_time", 0.1)

    # Define the optimization variables
    @variable(model, x[1:n_states, 1:N+1])
    @variable(model, u[1:n_inputs, 1:N])

    # Set the bounds on the variables
    u_min = -5.0
    u_max = 5.0
    x_min = [0.0, -30, 0.0, -30]
    x_max = [2*pi, 30, 2*pi, 30]
    @constraint(model, [i=1:N+1], x_min .<= x[:, i] .<= x_max)
    @constraint(model, [i=1:N], u_min .<= u[:, i] .<= u_max)

    # Define the initial state constraint
    @constraint(model, x[:, 1] .== x0)

    # Define the dynamics constraints
    for k in 1:N
        # @NLexpression(model, ψ, 1 / (α*β - γ^2 + (β^2 + γ^2)*sin(x[3, k])^2))
        @NLexpression(model, x1_next, x[2, k])
        @NLexpression(model, x2_next, (1 / (α*β - γ^2 + (β^2 + γ^2)*sin(x[3, k])^2))*(β*γ*(sin(x[3, k])^2-1)*sin(x[3, k])*x[2, k]^2 - 2*β^2*cos(x[3, k])*sin(x[3, k])*x[2, k]*x[4, k]
        + β*γ*sin(x[3, k])*x[4, k]^2 - γ*δ*cos(x[3, k])*sin(x[3, k]) + β*(-fϕ * x[2, k] + Ku * u[1, k]) - γ*cos(x[3, k])*(-fθ * x[4, k])))
        @NLexpression(model, x3_next, x[4, k])
        @NLexpression(model, x4_next, (1 / (α*β - γ^2 + (β^2 + γ^2)*sin(x[3, k])^2)) * (β*(α+β*sin(x[3, k])^2)*cos(x[3, k])*sin(x[3, k])*x[2, k]^2
        + 2*β*γ*(1-sin(x[3, k])^2)*sin(x[3, k])*x[2, k]*x[4, k] - γ^2*cos(x[3, k])*sin(x[3, k])*x[4, k]^2
        + δ*(α+β*sin(x[3, k])^2)*sin(x[3, k]) - γ*cos(x[3, k])*(-fϕ * x[2, k] + Ku * u[1, k]) + (α+β*sin(x[3, k])^2)*(-fθ * x[4, k])))
        @NLconstraint(model, x[1, k+1] == x[1, k] + dt * x1_next)
        @NLconstraint(model, x[2, k+1] == x[2, k] + dt * x2_next)
        @NLconstraint(model, x[3, k+1] == x[3, k] + dt * x3_next)
        @NLconstraint(model, x[4, k+1] == x[4, k] + dt * x4_next)

        unregister(model, :x1_next)
        unregister(model, :x2_next)
        unregister(model, :x3_next)
        unregister(model, :x4_next)

        # @NLconstraint(model, x[:, i+1] == x[:, i] + dt * dynamics(x[:, i], u[:, i]))
    end

    # Define the objective function
    @NLobjective(model, Min, sum(objective(x[1, i], x[2, i], x[3, i], x[4, i], u[1, i-1]) for i in 2:N+1))

    # Solve the optimization problem
    optimize!(model)

    # Extract the optimal control input
    u_opt = value.(u[:, 1])

    return u_opt
end

# Simulate the system dynamics
# function simulate_systems(ϕ::T, ϕdot::T, θ::T, θdot::T, u::T) where {T<:VariableRef}
#     # Use an appropriate numerical integration method to simulate the system dynamics
#     h = 0.002
#     dt = 0.01
#     steps = round(Int, dt / h)
#     for _ in 1:5
#         k1 = dynamics(ϕ, ϕdot, θ, θdot, u)
#         x = [ϕ, ϕdot, θ, θdot] + h * k1 / 3
#         k2 = dynamics(x[1], x[2], x[3], x[4], u)
#         x = [ϕ, ϕdot, θ, θdot] + h * (-k1 / 3 + k2)
#         k3 = dynamics(x[1], x[2], x[3], x[4], u)
#         x = [ϕ, ϕdot, θ, θdot] + h * (k1 - k2 + k3)
#         k4 = dynamics(x[1], x[2], x[3], x[4], u)
#         ϕ, ϕdot, θ, θdot = [ϕ, ϕdot, θ, θdot] .+ h .* (k1 .+ 3 .* k2 .+ 3 .* k3 .+ k4) ./ 8
#     end
#     # ϕ = mod2pi(ϕ)
#     # θ = mod2pi(θ)
#     return [ϕ, ϕdot, θ, θdot]
# end



nmpc_controller (generic function with 1 method)

In [24]:
# u_opt = nmpc_controller(x0, 5, 0.01)
@show u_opt

u_opt = [1.4474725041618445e-8]


1-element Vector{Float64}:
 1.4474725041618445e-8

In [26]:
# Call the NMPC controller
env = FurutaEnv()
N = 10
dt = 0.01
tmax = 5.
tspan = 0.:env.dt:tmax
xs = zeros(length(tspan),4)
for (i,t) in enumerate(tspan)
    x0 = measure(env.furuta)
    xs[i,:] = x0
    u_opt = nmpc_controller(x0, N, dt)
    env(u_opt[1])
end

include("../src/sim/env.jl")
animate_pendulum(xs[:,1],xs[:,3],tmax)

In [1]:
function dynamics(x, u)
    u = u[1]
    J = 1.54e-4
    M = 0
    ma = 0
    mp = 5.44e-3
    la = 4.3e-2
    lp = 6.46e-2
    fϕ = 0.0238
    fθ = 0.000008
    R = 0.13
    Ka = 0.03
    h = 0.01
    g = 9.81
    α = J+(M+ma/3+mp)*la^2
    β = (M+mp/3)*lp^2
    γ = (M+mp/2)*la*lp
    δ = (M+mp/2)*g*lp
    Ku = Ka / R

    # Base angle/speed, arm angle/speed
    ϕ, ϕdot, θ, θdot = x

    # clamp(Ku * p.u, -p.params.max_torque, p.params.max_torque)
    τϕm = Ku * u # Torque from motor (approximating that transients between voltage and current are fast)
    τϕf = -fϕ * ϕdot
    τϕ = τϕm + τϕf
    # theta joint
    τθ = -fθ * θdot

    ψ = 1 / (α*β - γ^2 + (β^2 + γ^2)*sin(θ)^2)
    dϕ    = ϕdot
    dϕdot = ψ*(β*γ*(sin(θ)^2-1)*sin(θ)*ϕdot^2 - 2*β^2*cos(θ)*sin(θ)*ϕdot*θdot
        + β*γ*sin(θ)*θdot^2 - γ*δ*cos(θ)*sin(θ) + β*τϕ - γ*cos(θ)*τθ)
    dθ    = θdot
    dθdot = ψ * (β*(α+β*sin(θ)^2)*cos(θ)*sin(θ)*ϕdot^2
        + 2*β*γ*(1-sin(θ)^2)*sin(θ)*ϕdot*θdot - γ^2*cos(θ)*sin(θ)*θdot^2
        + δ*(α+β*sin(θ)^2)*sin(θ) - γ*cos(θ)*τϕ + (α+β*sin(θ)^2)*τθ) 

    return [dϕ, dϕdot, dθ, dθdot]
end

dynamics (generic function with 1 method)

In [2]:
using ReinforcementLearning
using Flux
using Flux.Losses
using IntervalSets
using StableRNGs
using Plots

mutable struct MPCEnv <: AbstractEnv
    state
    reward::AbstractFloat
    action_space
    action::AbstractFloat
    state_space
    done
    furuta
    t
    dt
    tmax
    last_time
end

function MPCEnv(;
    max_u=5.,
    max_dθ=100.,
    max_dϕ=100.,
    dt = 0.01,
    tmax = 5.
    )
high = [Inf,max_dθ,Inf,max_dϕ]
low = [-Inf,-max_dθ,-Inf,-max_dϕ]
furuta = SimulatedFurutaPendulum()

MPCEnv(
    furuta.x,
    0.,
    ClosedInterval.(-max_u,max_u),
    0.,
    Space(ClosedInterval{Float64}.(low, high)),
    false,
    furuta,
    0.,
    dt,
    tmax,
    0
    )
end

RLBase.action_space(env::MPCEnv) = env.action_space
RLBase.state_space(env::MPCEnv) = env.state_space
function RLBase.reward(env::MPCEnv)
    ϕ, ϕdot, θ, θdot = x
    env.reward = - sum(objective(x[:, i], u[:, i]) for i in 1:N)  # Define the cost function
end
RLBase.is_terminated(env::MPCEnv) = env.done
RLBase.state(env::MPCEnv) = env.state

function (env::MPCEnv)(a::AbstractFloat)
env.action = a
control(env.furuta,a)
periodic_wait(env.furuta,env.t,env.dt)
env.state[:] = measure(env.furuta)
env.t += env.dt
env.done = env.t >= env.tmax
end

function RLBase.reset!(env::MPCEnv)
env.action = 0.
env.reward = 0.
env.t = 0.
env.last_time = 0.
env.furuta = SimulatedFurutaPendulum()
env.state = measure(env.furuta)
env.done = false
end

In [3]:
mutable struct FurutaEnv <: AbstractEnv
    state
    reward::AbstractFloat
    action_space
    action::AbstractFloat
    state_space
    done
    furuta
    t
    dt
    tmax
end

function FurutaEnv(;
        max_u=5.,
        max_dθ=100.,
        max_dϕ=100.,
        dt = 0.01,
        tmax = 10.
        )
    high = [Inf,max_dθ,Inf,max_dϕ]
    low = [-Inf,-max_dθ,-Inf,-max_dϕ]
    furuta = SimulatedFurutaPendulum()
    
    FurutaEnv(
        furuta.x,
        0.,
        ClosedInterval.(-max_u,max_u),
        0.,
        Space(ClosedInterval{Float64}.(low, high)),
        false,
        furuta,
        0.,
        dt,
        tmax
        )
end

RLBase.action_space(env::FurutaEnv) = env.action_space
RLBase.state_space(env::FurutaEnv) = env.state_space
function RLBase.reward(env::FurutaEnv)
#     x = env.furuta.x
#     ϕ, ϕdot, θ, θdot = [x[1],x[2],rem2pi(x[3],RoundNearest),x[4]]
    env.reward = -abs(0.0018522 - 0.0018522 * (cos(env.state[3])) - 0.00000387 * env.state[4]^2)
end
RLBase.is_terminated(env::FurutaEnv) = env.done
RLBase.state(env::FurutaEnv) = env.state

function (env::FurutaEnv)(a::AbstractFloat)
    @assert a in env.action_space
    env.action = a
    dt = env.dt
    control(env.furuta,a)
    periodic_wait(env.furuta,env.t,dt)
    env.state[:] = measure(env.furuta)
    env.t += dt
    env.done = env.t >= env.tmax
    nothing
end

function RLBase.reset!(env::FurutaEnv)
    env.action = 0.
    env.reward = 0.
    env.t = 0.
    env.furuta = SimulatedFurutaPendulum()
    env.state = env.furuta.x
#     env.state[3] = 0.
    env.done = false
end